In [1]:
import pandas as pd
import numpy as np
from itertools import product
from scipy.signal import convolve2d

### Reading initial seating input.

In [2]:
seating = pd.read_csv("./input/Day-11.txt",header=None)[0].str.extractall('(.)')[0].unstack().values
# Convert char representation: 'L' = empty (-1), '.' = floor (0), '#' occupied (1).
seating = (seating == '#') * 1 - (seating == 'L') * 1

### Generic solver for the stable seating problem

In [3]:
def solve_stable_seating(seating, adjacency_matrix_calculator, occupied_tolerance):
    """ Solves the stable seating problem, returning the total number of occupied seats. """
    seat_changes = np.array([1])
    seating = np.copy(seating)
    while seat_changes.any():
        adjacency_matrix = adjacency_matrix_calculator(seating) 
        seat_changes = (  (seating == -1) * (adjacency_matrix == 0)                  *  2   # New seats taken (-1 => 1)
                        + (seating == 1)  * (adjacency_matrix >= occupied_tolerance) * -2)  # Releasing occupied seats (1 => -1)
        seating += seat_changes
    return (seating > 0).sum()

### Q1: Adjacency is based on all 8-directions with distance of 1.

In [4]:
convolution_matrix = np.array([[1,1,1], [1,0,1], [1,1,1]])
def adjacency_matrix_q1(seating):
    return convolve2d(seating > 0, in2=convolution_matrix, boundary='fill', fillvalue=0)[1:-1,1:-1]

%timeit     solve_stable_seating(seating, adjacency_matrix_calculator=adjacency_matrix_q1, occupied_tolerance=4)
print("Q1", solve_stable_seating(seating, adjacency_matrix_calculator=adjacency_matrix_q1, occupied_tolerance=4))

112 ms ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Q1 2164


### Q2: Adjacency is based on the first seen seat (free or occupied) in each of the 8 directions.

In [5]:
def adjacency_seat_q2(seating, row, col):
    return np.sum([
        1 == next((x for x in         np.flip( seating[:row,   col   ])     if x != 0), 0),  # UP
        1 == next((x for x in                  seating[row+1:, col   ]      if x != 0), 0),  # DOWN
        1 == next((x for x in         np.flip( seating[row,    :col  ])     if x != 0), 0),  # LEFT
        1 == next((x for x in                  seating[row,    col+1:]      if x != 0), 0),  # RIGHT
        1 == next((x for x in np.diag(         seating[row+1:, col+1:])     if x != 0), 0),  # BOTTOM-RIGHT
        1 == next((x for x in np.diag(np.rot90(seating[row+1:, :col  ], 1)) if x != 0), 0),  # BOTTOM-LEFT
        1 == next((x for x in np.diag(np.rot90(seating[:row,   :col  ], 2)) if x != 0), 0),  # TOP-LEFT
        1 == next((x for x in np.diag(np.rot90(seating[:row,   col+1:], 3)) if x != 0), 0),  # TOP-RIGHT  
    ])

def adjacency_matrix_q2(seating):
    return np.array([adjacency_seat_q2(seating, i, j) for i,j in product(range(seating.shape[0]), range(seating.shape[1]))])\
             .reshape([seating.shape[0],seating.shape[1]])

%timeit     solve_stable_seating(seating, adjacency_matrix_calculator=adjacency_matrix_q2, occupied_tolerance=5)
print("Q2", solve_stable_seating(seating, adjacency_matrix_calculator=adjacency_matrix_q2, occupied_tolerance=5))

2min 19s ± 10.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
Q2 1974
